In [2]:
import os
import numpy as np
import pandas as pd
import hickle


from PIL import Image

# http://stackoverflow.com/questions/15514593/importerror-no-module-named-when-trying-to-run-python-script
import sys
sys.path.append('C:/Anaconda/Lib/site-packages')

import cv2
import PIL
import time

# Set DATA directory
DATA_DIR = 'C:/Users/thoma/Documents/00GitHub/rbc_cnn/'
CSV_DIR = DATA_DIR + 'csv/'

# Set IMAGE directory: Keep off of GitHub
IMG_DIR = 'C:/Users/thoma/Documents/00GitHub/00_LOCAL_ONLY/00RbcCNN_Sln_Images/'

# read in cell labels from Cell Label Tool 
classes = pd.read_csv(DATA_DIR + 'dataset.csv', index_col=0, parse_dates=True)

# get rid of reject cells
classes = classes[classes.label != 'Reject']
classes = classes[classes.label != 'Abnormal']

# counts the number of cells in each calss
#print classes['label'].value_counts()

def csv_to_dataFrame(path):
    
    # output csv
    #fout=open(outputFile,"a")
    i = 0 

    for root, dirs, files in os.walk(path):
        frame = pd.DataFrame()
        list_ = []
        for file_ in files:
            csv_path = os.path.join(root, file_)
            df = pd.read_csv(csv_path,index_col=0, header=0)
            list_.append(df)
        frame = pd.concat(list_)
    frame = frame.reset_index(drop=True)
    return frame

In [3]:
df = pd.DataFrame(np.random.randint(0,10,size=(10, 4)), columns=list('ABCD'))

In [4]:
df = df.drop(df.index[3])

In [3]:
def overlap_cell_check(df, cell, second_cell):
    
    overlap_list = []

    if cell.x in range(second_cell.x-30, second_cell.x+30):
        if cell.y in range(second_cell.y-30, second_cell.y+30):
            if cell.image == second_cell.image:
                if cell.annotator != second_cell.annotator:
                    overlap_list.append(cell)
                    overlap_list.append(second_cell)
                    return overlap_list
                else:
                    return None
            else:
                return None
        else:
            return None
    else:
        return None

    return 

In [4]:
def fix_overlap(overlap_list):
    new_overlap = []
    for item in overlap_list:
        if item[0]['annotator'] == item[1]['annotator']:
            print "Error"
        elif item[0]['label'] == item[1]['label']:
            new_overlap.append(item[0])

        else:
            continue
    return new_overlap

In [5]:
def parse_dataFrame(df):
    """"
    Takes DataFrame from all of csv files and returns multiple DFs in a dictionary.
    The DFs returned include Tommy's cells, Rick's Cells, and Non-Overlapping Cells. The last
    includes the cells that are found to be overlapping by x, y, and image vectors. 
    """
    x = np.array(df['x'])
    y = np.array(df['y'])
    i = np.array(df['image'])
    a = np.array(df['annotator'])

    # create linkage table subtracting (x from x) and (y from y)
    dx = x[:, np.newaxis] - x
    dy = y[:, np.newaxis] - y
    
    # set images that do not equal each other to large value
    di = (i[:, np.newaxis] != i)*1e9
    da = (a[:, np.newaxis] != a)*1e9

    # set diagonal of linkage table to large number
    dx[np.diag_indices(len(dx))] = 1e9
    dy[np.diag_indices(len(dy))] = 1e9
    di[np.diag_indices(len(di))] = 1e9
    da[np.diag_indices(len(da))] = 1e9

    # get absolute values
    dx = np.abs(dx)
    dy = np.abs(dy)

    # sum vector of x, y, and image
    d = dx + dy + di + da

    # set variable = to min of 0 for vector summation
    b = d.min(0)

    # create array with zeros of length b 
    exclude = np.zeros(len(b))

    # iterate through index of length b 
    for i in range(len(b)):
        # if True/anything present at exclude[i]; skip
        if exclude[i]:
            continue
        if b[i] < 30:
            # add True to index position of exclude if d[i] < 30     
            exclude[(d[i] < 30).nonzero()] = True
    
    d = {}

    # create overlap frame
    df_overlap = df[exclude == True]
    df_overlap = df_overlap.reset_index(drop=True)
    d['overlap'] = df_overlap

    # create non_overlap frame
    df_nonoverlap = df[exclude == False]
    df_nonoverlap = df_nonoverlap.reset_index(drop=True)
    d['non_overlap'] = df_nonoverlap

    # create rick frame
    df_r = df_nonoverlap[df_nonoverlap.annotator == 'rick']
    df_r = df_r.reset_index(drop=True)
    d['rick'] = df_r

    # create tommy frame
    df_t = df_nonoverlap[df_nonoverlap.annotator == 'tommy']
    df_t = df_t.reset_index(drop=True)
    d['tommy'] = df_t

    return d


In [100]:
df = csv_to_dataFrame(CSV_DIR)

In [101]:
df

,annotator,image,label,pk,x,y
0,tommy,Ecchinos.jpg,Echinocyte,2591,254,1167
1,tommy,Ecchinos.jpg,Echinocyte,2592,686,978
2,tommy,Ecchinos.jpg,Echinocyte,2593,781,985
3,tommy,Ecchinos.jpg,Echinocyte,2594,741,815
4,tommy,Ecchinos.jpg,Echinocyte,2595,415,858
5,tommy,Ecchinos.jpg,Echinocyte,2596,371,739
6,tommy,Ecchinos.jpg,Echinocyte,2597,321,924
7,tommy,Ecchinos.jpg,Echinocyte,2598,457,973
8,tommy,Ecchinos.jpg,Elliptocyte,2599,364,2342
9,tommy,Ecchinos.jpg,Acanthocyte,2600,351,2581


In [90]:
np.array(df.y)

array([1167,  978,  985, ..., 2678, 2338, 1697], dtype=int64)

In [61]:
x = np.array(df['x'])
y = np.array(df['y'])
i = np.array(df['image'])


# create linkage table subtracting (x from x) and (y from y)
dx = x[:, np.newaxis] - x
dy = y[:, np.newaxis] - y

# set images that do not equal each other to large value
di = (i[:, np.newaxis] != i)*1e9

# don't look at annotator because I think relabeled cells are in df twice
#da = (a[:, np.newaxis] == a)*1e9

# set diagonal of linkage table to large number
dx[np.diag_indices(len(dx))] = 1e9
dy[np.diag_indices(len(dy))] = 1e9
di[np.diag_indices(len(di))] = 1e9


# sum vector of x, y, and image
d = np.sqrt(dx**2 + dy**2 + di**2)


# set variable = to min of 0 for vector summation
b = d.min(0)



In [130]:
exclude = np.zeros(len(b))

# iterate through index of length b 
for i in range(len(b)):
    # if True/anything present at exclude[i]; skip
    if exclude[i]:
        continue
        # add True to index position of exclude if d[i] < 30     
    exclude[(d[i] < 20).nonzero()] = True
        
# create overlap frame
df_overlap_newWay = df[exclude == True]
#df_overlap_newWay = df_overlap_newWay.reset_index(drop=True)

df_overlap_newWay['annotator'].value_counts()

rick     121
tommy     27
Name: annotator, dtype: int64

In [135]:
df[df.annotator == 'tommy']['annotator'].value_counts()

tommy    978
Name: annotator, dtype: int64

In [123]:
exclude.nonzero()

(array([  13,   86,  110,  116,  125,  126,  127,  129,  130,  131,  132,
         133,  136,  138,  139,  140,  142,  144,  145,  146,  147,  148,
         150,  152,  153,  154,  162,  193,  237,  248,  250,  251,  254,
         256,  257,  258,  259,  260,  262,  263,  265,  266,  267,  269,
         270,  272,  273,  274,  275,  278,  279,  280,  281,  284,  285,
         286,  287,  288,  289,  291,  292,  293,  294,  295,  296,  297,
         298,  301,  304,  305,  306,  316,  318,  319,  321,  322,  327,
         328,  329,  334,  335,  337,  338,  351,  352,  355,  357,  362,
         363,  366,  367,  368,  369,  373,  376,  377,  380,  381,  386,
         389,  399,  413,  503,  645,  762,  763,  766,  768,  784,  789,
         791,  802,  804,  807,  813,  815,  818,  843,  853,  855,  862,
         867,  869, 1059, 1061, 1070, 1107, 1117, 1124, 1177, 1180, 1210,
        1217, 1353, 1359, 1388, 1391, 1394, 1395, 1439, 1464, 1468, 1510,
        1516, 1552, 1582, 1588, 1592, 

In [118]:
 df.ix[(d[ix] < 41).nonzero()]

KeyError: 

In [129]:
n = 0
for ix in exclude.nonzero()[0]:
    a = df.ix[ix].annotator
    if a != 'tommy':
        continue
    if (df.ix[(d[ix] < 41).nonzero()].annotator == a).any():
        n += 1
print n

31


In [111]:
(d[126] < 2).nonzero()

(array([56], dtype=int64),)

In [112]:
df.ix[126], df.ix[56]

(annotator            rick
 image        Ecchinos.jpg
 label         Acanthocyte
 pk                   2870
 x                     319
 y                    2741
 Name: 126, dtype: object, annotator           tommy
 image        Ecchinos.jpg
 label         Acanthocyte
 pk                   2647
 x                     319
 y                    2740
 Name: 56, dtype: object)

In [106]:
df.ix[df['pk']==791]

,annotator,image,label,pk,x,y
1353,rick,752300839HV_001.jpg,Elliptocyte,791,734,2093


In [83]:
df[exclude == True].sort_values(by='image')

,annotator,image,label,pk,x,y
1353,rick,752300839HV_001.jpg,Elliptocyte,791,734,2093
2009,tommy,752301114HV_001.jpg,Echinocyte,2153,1730,1643
1929,tommy,752301114HV_001.jpg,Echinocyte,2072,416,2014
1917,rick,752900792HV_001.jpg,Spherocyte,2060,1445,658
2662,rick,ERR20160406063658_001.jpg,Schistocyte,1692,1720,1179
2661,rick,ERR20160406063658_001.jpg,Schistocyte,1691,2004,1109
126,rick,Ecchinos.jpg,Acanthocyte,2870,319,2741
503,tommy,Schistocyte_04192016_8.jpg,Schistocyte,3009,801,2707
329,rick,Schitos.jpg,Schistocyte,2789,827,1898
338,rick,Schitos.jpg,Schistocyte,2798,931,1075


In [72]:
df.ix[298]

annotator            rick
image        Schitos2.jpg
label         Schistocyte
pk                   2758
x                    1161
y                    1958
Name: 298, dtype: object

In [73]:
df.ix[192]

annotator           tommy
image        Schitos2.jpg
label         Schistocyte
pk                   2536
x                    1160
y                    1959
Name: 192, dtype: object

In [74]:
df.ix[193]

annotator           tommy
image        Schitos2.jpg
label         Schistocyte
pk                   2537
x                    1177
y                    1934
Name: 193, dtype: object

In [49]:
df.iloc[exclude.nonzero()]['annotator'].value_counts()

rick     122
tommy     27
Name: annotator, dtype: int64

In [63]:
df_overlap_newWay.sort_values(by='x')

,annotator,image,label,pk,x,y
125,rick,Ecchinos.jpg,Acanthocyte,2869,76,1629
2304,rick,ERR20160320123115_001.jpg,Schistocyte,1333,77,1222
2109,tommy,752200524HV_001.jpg,Schistocyte,2256,148,1055
389,rick,Schistos1.jpg,Schistocyte,2850,267,1299
2110,tommy,752200524HV_001.jpg,Dacrocyte,2257,268,1874
289,rick,Schitos2.jpg,Schistocyte,2749,285,723
2237,rick,ERR20160320123115_001.jpg,Target cell,1265,292,970
1949,tommy,752301114HV_001.jpg,Dacrocyte,2092,309,1964
399,rick,Schitos.jpg,Schistocyte,2905,317,1854
126,rick,Ecchinos.jpg,Acanthocyte,2870,319,2741


In [ ]:
# old method

In [308]:
def overlap_cell_check(df, cell, second_cell):
    
    overlap_list = []

    if cell.x in range(second_cell.x-30, second_cell.x+30):
        if cell.y in range(second_cell.y-30, second_cell.y+30):
            if cell.image == second_cell.image:
                if cell.annotator != second_cell.annotator:
                    overlap_list.append(cell)
                    overlap_list.append(second_cell)
                    return overlap_list
                else:
                    return None
            else:
                return None
        else:
            return None
    else:
        return None

    return 

In [16]:
def parse_dataFrame(df):
    "remove double labeled cells"
    i = 0
    for1 = 0
    for2 = 0
    missedByY = 0
    overlap_list = []
    non_overlap = []

    # sort df by x value
    df = df.sort_values(by=['x'], ascending=[True])
    df = df.reset_index(drop=True)

    for n in range(len(df)-1):

        if n >= len(df):
            break

        # select one cell to compare to rest of dataset
        cell = df.ix[n]
        
        start = n - 30
        stop = n + 30

        if 0 > n - 30:
            start = 0
        else:
            start = start

        if len(df) < stop:
            stop = len(df)
        else:
            stop = stop

        for s in range(start, stop):
            second_cell = df.ix[s]
            olc = overlap_cell_check(df, cell, second_cell)
            if olc != None:
                overlap_list.append(olc)
                df = df.drop(df.index[n])
                df = df.drop(df.index[s])
                df = df.reset_index(drop=True)
                i += 1
                #print "{} overlapping cells".format(i)
            else:
                continue

    overlap_df = pd.DataFrame(overlap_list)

        
            
    return df, overlap_df

# gets byte array from smear image using xywh coordinates 
def get_cropped_array(ind, dataframe, im):
    # im = cv2.imread(IMG_DIR + dataframe.image[ind])
    df = dataframe.ix[ind]
    label = df.label
    
    new_x = df.x - 30
    new_y = df.y - 30
    new_w = 60
    new_h = 60

    croppedCell = im[new_y:new_y+new_h, new_x:new_x+new_w]

    if croppedCell.shape == (60, 60, 3):
        return croppedCell, label

In [17]:
# remove cells with overlapping XY coordinates
df, overlap_df = parse_dataFrame(df)

In [23]:
overlap_df

,0,1
0,annotator tommy image ...,annotator rick image ...
1,annotator rick image Ecchino...,annotator tommy image Ecchino...
2,annotator tommy image ...,annotator rick image ...
3,annotator tommy image Schist...,annotator rick image Schist...
4,annotator tommy image ...,annotator rick image ...
5,annotator rick image Schitos...,annotator tommy image Schitos...
6,annotator rick image ...,annotator tommy image ...
7,annotator rick image Schitos....,annotator tommy image Schitos....
8,annotator tommy image Ecchino...,annotator rick image Ecchino...
9,annotator rick image ...,annotator tommy image ...


In [409]:
overlap_df_oldWay = np.concatenate((overlap_df[0], overlap_df[1]))
overlap_df_oldWay = pd.DataFrame(overlap_df_oldWay)
overlap_df_oldWay

,0
0,annotator tommy image ...
1,annotator rick image Ecchino...
2,annotator tommy image ...
3,annotator tommy image Schist...
4,annotator tommy image ...
5,annotator rick image Schitos...
6,annotator rick image ...
7,annotator rick image Schitos....
8,annotator tommy image Ecchino...
9,annotator rick image ...
